In [3]:
#import libs
import matplotlib.pyplot as plt
import numpy as np
import pyaudio
import struct
import os
from scipy.fftpack import fft
from tkinter import TclError


%matplotlib tk 
#opens up new window for live graphing

CHUNK = 2048 #size of each audio packet
FORMAT = pyaudio.paInt16 #bit size of audio packet input
CHANNELS = 1 #mono channel microphone
RATE = 44100 #44.1 kHz recording freq

In [6]:
#INITIALIZE PYAUDIO STREAM
PAud = pyaudio.PyAudio()
stream = PAud.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

#SET UP PLOTS
fig, (ax, ay) = plt.subplots(2, figsize=(15,8))

#X-Axes
Audx = np.arange(0, 2*CHUNK, 2)
FFTx = np.linspace(0, RATE, CHUNK)

#Line objs
Audline, = ax.plot(Audx, np.random.rand(CHUNK), '-')
FFTline, = ay.semilogx(FFTx, np.random.rand(CHUNK), '-')

#make the plot look nice
ax.set_title('Recorded Audio')
ax.set_ylabel('Magnitude')
ax.set_ylim(0,255)
ax.set_xlim(0,2*CHUNK)
plt.setp(ax, xticks=[0, CHUNK, 2*CHUNK], yticks=[0,128,255])
ay.set_xlim(20, RATE/2)


#LOOP FOR AUDIO
#THE 'DO STUFF' PART OF THE CODE
while True:
    #Gather data and unpack into integers
    data = stream.read(CHUNK)
    Idata=struct.unpack(str(2*CHUNK) + 'B', data)
        
    #place into array and offset accordingly for smooth sound wave
    Ndata = np.array(Idata, dtype='b')[::2] + 128
    Audline.set_ydata(Ndata)
        
    #do the FFT magic
    FFTy = fft(Ndata)
    FFTline.set_ydata(np.abs(FFTy[0:CHUNK]) * 4 / (256 * CHUNK))
        
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
            
    except TclError:
        print ('Goodbye!')
        break

Goodbye!
